In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from urllib.parse import urljoin
import time
import random
import json
import csv


# 设置谷歌驱动器的环境
options = webdriver.ChromeOptions()
# 设置chrome不加载图片，提高速度
options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
# 创建一个谷歌驱动器
browser = webdriver.Chrome(options=options)
url = 'http://wap.cnki.net/touch/web/guide'

# 声明一个全局列表，用来存储字典
data_list = []


def start_spider(page):
    # 请求url
    browser.get(url)
    # 显示等待输入框是否加载完成
    WebDriverWait(browser, 1000).until(
        EC.presence_of_all_elements_located(
            (By.ID, 'keyword')
        )
    )
    # 找到输入框的id，并输入python关键字
    browser.find_element_by_id('keyword').click()
    browser.find_element_by_id('keyword_ordinary').send_keys('Urban innovation capability OR Regional innovation capability')
    # 输入关键字之后点击搜索
    browser.find_element_by_class_name('btn-search ').click()
    # print(browser.page_source)
    # 显示等待文献是否加载完成
    WebDriverWait(browser, 1000).until(
        EC.presence_of_all_elements_located(
            (By.CLASS_NAME, 'g-search-body')
        )
    )

    # 声明一个标记，用来标记翻页几页
    count = 1
    while True:
        # 显示等待加载更多按钮加载完成
        WebDriverWait(browser, 1000).until(
            EC.presence_of_all_elements_located(
                (By.CLASS_NAME, 'c-company__body-item-more')
            )
        )
        # 获取加载更多按钮
        Btn = browser.find_element_by_class_name('c-company__body-item-more')
        # 显示等待该信息加载完成
        WebDriverWait(browser, 1000).until(
            EC.presence_of_all_elements_located(
                (By.XPATH, '//div[@id="searchlist_div"]/div[{}]/div[@class="c-company__body-item"]'.format(2*count-1))
            )
        )
        # 获取在div标签的信息，其中format(2*count-1)是因为加载的时候有显示多少条
        # 简单的说就是这些div的信息都是奇数
        divs = browser.find_elements_by_xpath('//div[@id="searchlist_div"]/div[{}]/div[@class="c-company__body-item"]'.format(2*count-1))
        # 遍历循环
        for div in divs:
            # 获取文献的题目
            name = div.find_element_by_class_name('c-company__body-title').text
            # 获取文献的作者
            author = div.find_element_by_class_name('c-company__body-author').text
            # # 获取文献的摘要
            # content = div.find_element_by_class_name('c-company__body-content').text
            # 获取文献的来源和日期、文献类型等
            text = div.find_element_by_class_name('c-company__body-name').text.split()
            if (len(text) == 3 and text[-1] == '优先') or len(text) == 2:
                # 来源
                source = text[0]
                # 日期
                datetime = text[1]
                # 文献类型
                literature_type = None
            else:
                source = text[0]
                datetime = text[2]
                literature_type = text[1]
            # 获取下载数和被引数
            temp = div.find_element_by_class_name('c-company__body-info').text.split()
            # 下载数
            download = temp[0].split('：')[-1]
            # 被引数
            cite = temp[1].split('：')[-1]

            # ----------2020-3-18修改----------#
            # 文献链接
            link = div.find_element_by_class_name('c-company-top-link').get_attribute('href')
            # 拼接
            link = urljoin(browser.current_url, link)
            # 获取关键字（需要访问该文献，url就是上面获取到的link）
            # browser.get(link) # 这行和下面那行不推荐使用，因为句柄的问题，会报错
            # browser.back()
            # 打印查看下未访问新窗口时的句柄
            # print(browser.current_window_handle)
            js = 'window.open("%s");' % link
            # 每次访问链接的时候适当延迟
            time.sleep(random.uniform(1, 2))
            browser.execute_script(js)
            # 打印查看窗口的句柄，对比看下当前的句柄是哪个
            # 结果是原先窗口的句柄，而不是新打开窗口的句柄，因为和上面打印的句柄一样
            # print(browser.current_window_handle)
            # 切换句柄到新打开的窗口，browser.window_handles是查看全部的句柄
            # browser.switch_to_window是切换句柄
            browser.switch_to_window(browser.window_handles[1])
            # 获取关键字（使用xpath）
            key_worlds = browser.find_elements_by_xpath('//div[@class="c-card__paper-name"][contains(text(), "关键词")]/following-sibling::div[1]/a')
            key_worlds = ','.join(map(lambda x: x.text, key_worlds))
            # ----------2020-3-19修改----------#
            # 获取文献的摘要
            content = browser.find_element_by_class_name('c-card__aritcle').text
            # ----------2020-3-19修改----------#
            # 获取信息完之后先关闭当前窗口再切换句柄到原先的窗口
            browser.close()
            browser.switch_to_window(browser.window_handles[0])
            # 注：切换句柄参考该文章，感谢该博主：https://blog.csdn.net/DongGeGe214/article/details/52169761
            # ----------2020-3-18修改----------#
            
            # 声明一个字典存储数据
            data_dict = {}
            data_dict['name'] = name
            data_dict['author'] = author
            data_dict['content'] = content
            data_dict['source'] = source
            data_dict['datetime'] = datetime
            data_dict['literature_type'] = literature_type
            data_dict['download'] = download
            data_dict['cite'] = cite
            data_dict['link'] = link
            data_dict['key_worlds'] = key_worlds

            data_list.append(data_dict)
            print(data_dict)
        # 如果Btn按钮（就是加载更多这个按钮）没有找到（就是已经到底了），就退出
        if not Btn:
            break
        else:
            Btn.click()
        # 如果到了爬取的页数就退出
        if count == page:
            break

        count += 1

        # 延迟两秒，我们不是在攻击服务器
        time.sleep(2)
    # 全部爬取结束后退出浏览器
    browser.quit()


def main():

    start_spider(eval(input('请输入要爬取的页数（如果需要全部爬取请输入0）：')))

    # 将数据写入json文件中
    with open('data_json.json', 'a+', encoding='utf-8') as f:
        json.dump(data_list, f, ensure_ascii=False, indent=4)
    print('json文件写入完成')

    # 将数据写入csv文件
    with open('data_csv.csv', 'w', encoding='utf-8', newline='') as f:
        # 表头
        title = data_list[0].keys()
        # 声明writer对象
        writer = csv.DictWriter(f, title)
        # 写入表头
        writer.writeheader()
        # 批量写入数据
        writer.writerows(data_list)
    print('csv文件写入完成')


if __name__ == '__main__':

    main()


请输入要爬取的页数（如果需要全部爬取请输入0）：6


D:\anaconda\lib\site-packages\ipykernel_launcher.py:113: DeprecationWarning: use driver.switch_to.window instead
D:\anaconda\lib\site-packages\ipykernel_launcher.py:123: DeprecationWarning: use driver.switch_to.window instead


{'name': 'Urban Hierarchy of Innovation Capability and Inter-city Linkage...', 'author': '', 'content': 'The 2000s has witnessed increasing interests in cities′role of innovation in the era of knowledge-based economy.Compared with substantial empirical analysis on the world city hierarchy of innovation,this paper attempts to examine the national urban hierarchy of innovation capability in China,in terms of ranking systems,spatial pattern and inter-city linkages of knowledge during the post-reform period since the late 1970s.Based on quantitative analysis such as principal component factor analysis and clustering analysis,this paper identifies the five-tier hierarchy of in-novation,which is headed by Beijing and Shanghai,followed by the capital cities of each province and regional centre cities.The development of China′s urban hierarchy of innovation capability has been driven by such factors as the scale of innovation,scientific scale,innovation potential and innovation environment.The

{'name': '金融危机下的中国区域创新能力——中国31个省区市创新能力指数2008年实证和2009年...', 'author': '赵彦云 甄峰 吴翌琳 王敏', 'content': '中国正在全面奔向创新型国家的建设之中,区域创新能力提升是根本,创新指数依据知识生产函数理论,结合我国自主创新目标和过程,以及非技术创新的特殊性,全面刻画我国区域创新能力。2008年世界金融危机对我国经济产生了比较大的影响,但是我国区域创新能力仍持续发展,2009年将进一步强化创新能力对经济发展的作用。', 'source': '经济理论与经济管理', 'datetime': '核心', 'literature_type': '2009年08期', 'download': '1328', 'cite': '19', 'link': 'https://wap.cnki.net/touch/web/Journal/Article/JJLL200908012.html', 'key_worlds': '创新指数,区域创新能力,聚类分析'}
{'name': '中国区域创新能力:因素分析与聚类研究——兼论区域创新能力综合评价的因素分析替...', 'author': '周立 吴玉鸣', 'content': '基于《中国区域创新能力报告》中的数据,采用因素分析与聚类分析相结合的综合集成评估方法,对我国大陆31个省级区域的创新能力进行了定量评估及比较,同时探讨了因素分析法对加权综合评价方法的替代技术。结果显示,中国省域创新能力的地区差异比较明显,用因素分析法替代现有的区域创新能力综合评价是可行的,必须分集团而非整齐划一地制定和实施增强区域自主创新能力的对策建议。', 'source': '中国软科学', 'datetime': '核心', 'literature_type': '2006年08期', 'download': '5190', 'cite': '246', 'link': 'https://wap.cnki.net/touch/web/Journal/Article/ZGRK200608011.html', 'key_worlds': '区域创新能力,加权综合评价,因素分析,聚类分析'}
{'name': '全面创新改革试验对区域创新能力的影响效应与传导机制', 'auth

{'name': '创新环境对促进我国区域创新能力的影响', 'author': '党文娟 张宗益 康继军', 'content': '我国经济发展的不平衡导致各个地区的市场化进程、政府干预程度存在很大差异,区域环境很不平衡。本文运用全国各个地区的数据,采用计数模型中的负二项分布negatitive-binomial方法分析了区域环境对促进区域创新能力的影响。结果表明我国政府对提高区域整体创新能力的影响作用还不明显,但是对具有自主创新能力和增强原创性的发明专利却具有积极的促进作用。各个地区的市场化程度对提高区域创新能力具有非常明显的促进作用,区域环境对区域创新能力具有非常重要的影响作用。只有积极进行区域市场化程度的推进以及发挥各地政府的创新引导作用才是提高区域创新能力的关键。', 'source': '中国软科学', 'datetime': '核心', 'literature_type': '2008年03期', 'download': '2622', 'cite': '206', 'link': 'https://wap.cnki.net/touch/web/Journal/Article/ZGRK200803007.html', 'key_worlds': '市场化程度,政府干预程度,区域创新能力,计数模型'}
{'name': '建设区域创新体系 提高区域创新能力', 'author': '吴丽鸣', 'content': '在新的历史时期,面对全面建设小康社会的客观要求,面对市委提出的“一争一创一率先”的奋斗目标,要把握我市科技工作的重点,有效地促进区域创新体系建设,切实提高区域创新能力,努力为全市经济社会持续健康发展做出贡献。', 'source': '济源职业技术学院学报', 'datetime': '2003年03期', 'literature_type': None, 'download': '59', 'cite': '1', 'link': 'https://wap.cnki.net/touch/web/Journal/Article/JYZB200303000.html', 'key_worlds': '区域创新,体系,能力,建设,提高'}
{'name': '珠三角城市创新能力及创新空间溢出效应研究', 'author': '张璇', 'co

{'name': '基于制度环境门槛模型的研发投入对区域创新能力提升研究', 'author': '吾买尔江·艾山 郑惠 孙春阳', 'content': '新常态背景下,我国经济增长引擎转向创新驱动,加大研发投入成为快速提高自主创新能力的重要途径。文章运用我国2005—2016年省际面板数据,以地区腐败、市场化进程与知识产权保护水平作为制度环境的代理变量,考察研发资金投入、研发人员投入对区域创新能力影响的门槛效应。研究结果表明:研发投入对区域创新能力的影响具有明显的制度环境双门槛效应:在地区腐败较低时,研发资金投入、研发人员投入对区域创新能力的促进作用较大,且二者的弹性系数均随着地区腐败程度的增加而减小;在市场化进程较低的地区,研发资金投入与研发人员投入对区域创新能力具有较小的正向促进作用,随着市场化进程的不断提升,研发资金投入与研发人员投入的弹性系数不断增大;在知识产权保护水平较低的地区,研发资金投入与研发人员投入对区域创新能力具有较小的正向促进作用,随着地区知识产权保护水平的提升,研发资金投入与研发人员投入对区域创新能力的促进作用逐渐增大。因此,在利用研发投入促进创新能力提升的过程中,各地区应充分考虑为其创造良好的制度环境。', 'source': '新疆大学学报(哲学·人文社会科学版)', 'datetime': 'CSSCI', 'literature_type': '2020年03期', 'download': '530', 'cite': '1', 'link': 'https://wap.cnki.net/touch/web/Journal/Article/XJDB202003002.html', 'key_worlds': '制度环境,研发投入,区域创新能力,门槛效应'}
{'name': '互联网发展与区域创新能力非线性关系研究', 'author': '惠宁 刘鑫鑫', 'content': '党的十九大报告要求全力推动互联网、大数据、人工智能与实体经济深度融合。在此背景下,明晰互联网与区域创新能力关系至关重要。基于2006-2017年中国内地30个省区市面板数据,运用门槛面板回归技术,全面探讨了互联网发展与区域创新能力间的非线性关系。研究结果表明:互联网发展对区域创新能力存在边际报酬递增的正向非线性影响,且在政府支持环境下两者存在正向、边际报

{'name': '交通基础设施、人力资本对区域创新能力影响的实证研究', 'author': '马明 薛晓达 赵国浩', 'content': '使用1995—2015年中国30个省区面板数据,在空间杜宾面板模型基础上加入时间因素,构建动态空间杜宾面板模型,使用经济距离和经济地理距离空间权重矩阵,实证分析了交通基础设施与人力资本两因素对区域创新能力的空间溢出效应。结果显示:(1)两因素在经济距离空间权重矩阵和经济地理距离空间权重矩阵中均对区域创新能力产生了显著的空间溢出效应和时间累积效应。前者对区域创新能力产生正空间溢出,后者对区域创新能力产生负空间负溢出;(2)两因素对区域创新能力空间溢出效应的大小同时受地理距离与经济发展水平的影响;(3)两因素对区域创新能力速度提升的影响不显著。', 'source': '北京理工大学学报(社会科学版)', 'datetime': '核心', 'literature_type': '2018年01期', 'download': '1442', 'cite': '36', 'link': 'https://wap.cnki.net/touch/web/Journal/Article/BLDS201801012.html', 'key_worlds': '交通基础设施,人力资本,区域创新能力,动态空间杜宾面板模型'}
{'name': '城市化、工业化与区域创新能力——基于空间计量模型的实证分析', 'author': '付琦 卓乘风 邓峰', 'content': '在实施创新驱动战略背景下,区域创新能力是推动区域经济发展的重要力量。在考虑空间因素的基础上,选取2005—2015年中国省域面板数据,分析城市化、工业化发展对区域创新能力的作用机制。实证结果表明:城市化、工业化水平的提高对区域创新能力具有正向促进作用,已成为推动区域创新发展的源泉;在考虑城市化与工业化的交互作用时,城市化和工业化对区域创新能力的促进效应有一定的提升。', 'source': '科技管理研究', 'datetime': '核心', 'literature_type': '2018年01期', 'download': '768', 'cite': '12', 'link': 'https://wap.cnki.net/touch/web/Journal/

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jun 12 09:14:57 2020
@author: 13284
"""
# worcloud是生成词云的关键
import wordcloud
# numpy是一个强大的科学计算库,我们将用它来把PIL库打开的图片保存为数组
import numpy as np
# PIL是一个图像处理库,用来打开图片
from PIL import Image
# jieba是中文分词库,可以智能地拆分词语
import jieba
# matplotlib库是一个绘图库,可以用来显示图片以及绘制多种图形
import matplotlib.pyplot as plt
# 生成背景图片的两种方法#
# 注意背景图片的路径要写对，可以是完整路径，例如C:/image/test.jpg
# 若图片与Python代码在同一个文件夹，那可以直接写图片名称+后缀，例如test.jpg
# 1.使用 matplotlib库读取图片
# background_image_1.jpg是背景图片的文件名，要替换为你自己的图片
#bg_1=plt.imread("background_image_1.jpg")
# 2.使用numpy库和PIL库，PIL库用于打开图片，numpy库用于将图片转化为数组
# 下面这行代码我注释掉了
# bg_2=np.array(Image.open('background_image_2.jpg'))
# 对于以上的图片选择，要特别注意图片的背景应该为白色
#配置词云的背景，图片，字体大小等参数，
# 也可以不配置，直接 wc=wordcloud.WordCloud()
wc=wordcloud.WordCloud(
  # font_path为字体文件的路径，如果没有这个，那么生成的词云图片无法显示中文
  font_path="C:/Windows/Fonts/simfang.ttf",
  # scale为按比例放大或者缩小生成的图片。例如1.5表示图片放大为原来的1.5倍，可省略
  scale=1.5,
  # mask表示背景图片，如果不没有背景图片，那这个可以省略
  #mask=mask,
  # background_color为背景颜色，默认为黑色，可以省略
  background_color="white",
  # width为词云生成的图片宽度，默认为400
  width=600,
  # height为词云生成的图片高度，默认为200
  height=400,
  # max_words图片上显示的最大词语的个数
  max_words=1000,
  # max_font_size为最大字体的大小
  max_font_size=100,
  # min_font_size为最小字体大小,默认为4
  min_font_size=8)
# 打开要生成词云的文件,一般是txt格式的纯文本文档
# 词云来源.txt 要替换成存有你要转化为词云的信息文件的文件名
with open(r'C:\Users\13284\Desktop\大三下期末大整理\111.txt','r',encoding="utf-8") as f:
  # text=f.read()是说读取 词云来源.txt 这个文件的所有内容并存储到变量text中
  text=f.read()
  # 利用jieba库把text拆分并且使用空格分隔拆分后的结果
  
print("正在生成词云......")
# 调用generate方法，传入要生成词云的文本即可生成词云
wc.generate(text)
# 把词云保存成图片
wc.to_file('wordcloud.jpg')
print("生成完毕!")

TypeError: argument should be integer or None, not 'str'